# Loading/Processing Corpus

## Issues:

* For finding nuggets, get sent_id from update_id (maybe from matches file) and +1 that number
* Try alternative using char matching from start, end columns. See below in test func how. Can use Spacy or splitlines sentences and char match with their lens
* Maybe add matches to filepath sources?
* Functionality for multiple nuggets/matches source files

In [16]:
from bs4 import BeautifulSoup as bs
import gzip
import pandas as pd
import numpy as np
from tqdm import tqdm
from tqdm import tqdm_notebook
# from tqdm.notebook import tqdm
import os
from collections import OrderedDict
import pickle
import warnings
import copy
import spacy
from sentence_transformers import SentenceTransformer

### Meta File Variables

In [5]:
corpus_path = "/nfs/trects-kba2014-filtered" # directory of corpus of gzipped html files
topics_path = corpus_path + "/test-topics.xml"
doc_tags = ['topic_id','streamid', 'docid', 'yyyymmddhh', 'kbastream', 'zulu', 'epoch', 'title', 'text', 'url'] # doc fields
topic_tags = ['id', 'title', 'description', 'start','end','query','type'] # topic fields
test_file_addr = corpus_path + "/1/2012-02-22-15.gz"
proj_dir = '/nfs/proj-repo/AAARG-dissertation'
csv_dir = proj_dir + '/' + 'load_data'
# csv file addresses
corp_csv_name = 'corpus_loaded.csv.gz'
corp_csv_path = csv_dir + '/' + corp_csv_name
topics_csv_name = 'topics_loaded.csv.gz'
topics_csv_path = csv_dir + '/' + topics_csv_name
# nugget/update dataframes
nugget_dir = "/nfs/TemporalSummarization/ts13/results"
updates_sampled_path = nugget_dir + "/updates_sampled.tsv"
nuggets_path = nugget_dir + "/nuggets.tsv"
nug_matches_path = nugget_dir + "/matches.tsv"
# saving nugget and update files
nugget_csv = 'nugget_df.csv.gz'
update_csv = 'update_df.csv.gz'
nugget_csv_path = csv_dir + '/' + nugget_csv
update_csv_path = csv_dir + '/' + update_csv
# supervised input/labels
# supervised_csv = 'supervised_df.csv.gz'
# supervised_csv_path = csv_dir + '/' + supervised_csv

In [249]:
def convert_to_list(item):
    if type(item) is not list:
        item = [item]
    return item

def file_exists(path):
    """Check if path or list of paths has item that does not exist"""
    exists = []
    path = convert_to_list(path)
    for p in path:
        exists.append(os.path.exists(p))
    return all(exists)

In [272]:
class FilePathHandler:
    """Paths will be in the format:
    
    proj_dir/dataset_dir/corpus_name/file_purpose/instance_identifier+split_identifier+sfile_type
    """
    def __init__(self, proj_dir, dataset_dir="dataset", compression='gzip'):
        self.proj_dir = proj_dir
        self.dataset_dir = proj_dir + '/' + dataset_dir
        self.create_dir_if_not_exists(self.dataset_dir)
        self.path_df_path = self.dataset_dir + '/' + 'file_path_df.csv.gz'
        self.corpus_sources_pickle_path = self.dataset_dir + '/' + 'corpus_sources.pickle'
        self.compression = compression
        self.file_purposes = ["topics", "corpus", "nuggets", "embed_labels", "updates"]
#         self.path_df_cols = ["corpus_name", "file_purpose", "split_identifier", "num_splits",
#                             "instance_identifier", "file_type", "path", "exists"]
        self.path_df_cols = {"corpus_name":str, "file_purpose":str, "split_identifier":str, "num_splits":int,
                            "instance_identifier":str, "file_type":str, "path":str, "exists":bool}
        self.corpus_sources_keys = ['corpus_name', 'dir_path', 'nuggets_path', 'matches_path', 'topics_path']
        
        # load meta files
        self.load_corpus_sources()
        self.load_path_df()


    def paths_in_corpus_name(self, corpus_name, selection=None):
        def get_identity_paths(ident_df, exists=True):
            ident_paths = list(ident_df[ident_df['exists']==exists]['relative_path'])
            if len(ident_paths) == 0:
                ident_paths = None
            return ident_paths
        
        if selection is None:  # get all file purposes if none selected
            selection = self.file_purposes
        
        paths = {}
        name_df = self.path_df[self.path_df['corpus_name'] == corpus_name]
        for file_purpose in selection:  # loop for each file purpose/stage of generation
            purp_df = name_df[name_df['file_purpose'] == file_purpose]
            idents = purp_df['instance_identifier'].unique()  # segregate any unique identifiers/instances
            if len(idents) == 0:
                # case where corpus_name or file_purpose not in paths_df
                # will create same dict with None entries
                idents.append(None)  
            for ident in idents:
                ident_df = purp_df[purp_df['instance_identifier'] == ident]
                paths[file_purpose][ident]['exists'] = get_identity_paths(ident_df, exists=True)
                paths[file_purpose][ident]['not_exists'] = get_identity_paths(ident_df, exists=False)
        return paths
        
    
    def get_path(self, corpus_name, file_purpose, inst_identifier, file_type, add_path=True, exists=False,
                split_identifier=None, num_splits=None, warn=False):
        # do check here make sure filename compatible, or elsewhere
        path = self.dataset_dir + '/' + corpus_name + '/' + file_purpose + '/' + str(inst_identifier)
        if split_identifier is not None:
            path += '_' + str(split_identifier)
        path += file_type
        
        if add_path:
            self.add_path_to_df(corpus_name, file_purpose, split_identifier, num_splits, inst_identifier,
                            file_type, path, exists, save=True, warn=warn)
        return path
            
    def add_path_to_df(self, corpus_name, file_purpose, split_identifier, num_splits, inst_identifier,
                       file_type, path, exists, save=True, warn=False):
        if not (self.path_df['path'] == path).any():  # check if row exists
            # create appropriate dir if needed
            new_dir_path = self.dataset_dir + '/' + corpus_name + '/' + file_purpose
            self.create_dir_if_not_exists(new_dir_path)
            # add to path_df
            if num_splits is None:
                num_splits = 1
            
            row = pd.DataFrame({'corpus_name': pd.Series([corpus_name], dtype=str),
                                  'file_purpose': pd.Series([file_purpose], dtype=str),
                                  'split_identifier': pd.Series([split_identifier], dtype=str),
                                  'num_splits': pd.Series([num_splits], dtype=int),
                                  'instance_identifier': pd.Series([inst_identifier], dtype=str),
                                  'file_type': pd.Series([file_type], dtype=str),
                                  'path': pd.Series([path], dtype=str),
                                  'exists': pd.Series([exists], dtype=bool)})
            
            self.path_df = self.path_df.append(row, ignore_index=True)
            if save:  # save new path_df
                self.save_path_df()
        else:
            if warn:
                warnings.warn("Path already exists in dataframe: " + str(path))
            
    def update_path_exists(self, path, save=True):
        self.path_df.loc[self.path_df['path'] == path, 'exists'] = True
        if save:
            self.save_path_df()
        

    def create_dir_if_not_exists(self, dir_path, warn=True):
        if not os.path.exists(dir_path):
            os.makedirs(dir_path)
            warnings.warn("Created new directory at " + str(dir_path))
            return True
        return False
    
    def search_path_df(self, search_dict, df_slice=None):
        if df_slice is None:
            df_slice = self.path_df
        for col_name, value in search_dict.items():
            df_slice = df_slice[col_name == value]
        return df_slice
    
    def source_dict_correct(self, source_dict):
        # check has all appropriate keys
        for key in self.corpus_sources_keys:
            if key not in source_dict:
                raise Exception(str(key) + " is missing from corpus_source dict")
        false_paths = []
        for path_type, path in source_dict.items():
            if path_type == "corpus_name":  # dict entry not a path, don't check
                continue
            if not file_exists(path):
                false_paths.append(str(path_type) + " does not exist at " + str(path))
        if len(false_paths) > 0:
            error_str = "\n".join(false_paths)
            raise FileNotFoundError(error_str)
        else:
            return True
        
    
    def create_corpus_source_dict(self, corpus_name, dir_path, topics_file_path, nuggets_file_path,
                                 matches_file_path):
        s_dict = {"corpus_name":corpus_name, "dir_path":dir_path, 
                  "topics_path":topics_file_path, "nuggets_path":nuggets_file_path,
                 "matches_path":matches_file_path}
        self.source_dict_correct(s_dict)
        return p_dict
    
    def add_corpus_source(self, corpus_source_dict, overwrite=False):
        """Add a corpus directory to load from and its meta files"""
        # check paths exist
        self.source_dict_correct(corpus_source_dict)
        corpus_name = copy.deepcopy(corpus_source_dict["corpus_name"])
        # store new entry
        if corpus_name in self.corpus_sources and overwrite==False:
            warnings.warn(str(corpus_name) + " is already present in corpus source dictionary. \n Proceeding with dict entry")
        else:
            del corpus_source_dict["corpus_name"]  # remove corpus_name from dict to add as a key
            self.corpus_sources[corpus_name] = corpus_source_dict
            # create folder for outputting new source files
            new_corpus_dir = self.dataset_dir + '/' + corpus_name
            self.create_dir_if_not_exists(new_corpus_dir)
            self.save_corpus_sources()
            
    def get_corpus_sources(self, corpus_names=None):
        """Retrieve file paths from corpus_load dicts
        Parameters:
            corpus_names: list of corpus names retrieve, if None then retrieve all
        
        Returns:
            A dictionary where keys are the corpus names and values are target file paths
        """
        if corpus_names is None:
            corpus_names = self.corpus_sources.keys()
        corpus_paths = {}
        for name in corpus_names:
            corpus_paths[name] = self.corpus_sources[name]
        return corpus_paths
    
    def save_corpus_sources(self):
        with open(self.corpus_sources_pickle_path, 'wb') as handle:
            pickle.dump(self.corpus_sources, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    def load_corpus_sources(self):
        if os.path.exists(self.corpus_sources_pickle_path):
            with open(self.corpus_sources_pickle_path, 'rb') as handle:
                self.corpus_sources = pickle.load(handle)
            return True
        else:
            self.corpus_sources = {} # create empty dictionary
            return False
        
    def save_path_df(self):
#         self.force_path_df_col_type()
        self.path_df.to_csv(self.path_df_path, compression=self.compression)
        
    def load_path_df(self):
        """File containing info about file paths to systematically load files"""
        if os.path.exists(self.path_df_path):
            self.path_df = pd.read_csv(self.path_df_path, compression=self.compression)

            for col in self.path_df.columns:
                if "Unnamed" in col:
                    del self.path_df[col]
            return True
        else:
            path_df_dict = {}
            for col_name, data_type in self.path_df_cols.items():  # ensure columns don't infer wrong typing
                path_df_dict[col_name] = pd.Series([], dtype=data_type)
            self.path_df = pd.DataFrame(path_df_dict)  # create empty dataframe
#             self.force_path_df_col_type()
            return False
        
#     def force_path_df_col_type(self):
#         """stop path_df autoconverting ints to floats in these columns"""
#         # make sure these columns are type object
#         obj_cols = ["split_identifier", "num_split", "instance_identifier"]
#         for col in obj_cols:
#             if col in self.path_df.columns:
#                 self.path_df[col] = pd.Series([], dtype=object)  # handles Nones better

In [4]:
# test_path_f = FilePathHandler(proj_dir)
# del test_path_f.corpus_sources['tr14_init_filtered']
# print(test_path_f.corpus_sources)

In [5]:
# # def supervised_path_generator(identifier, base="supervised_df", handle=".csv.gz"):
# #     fn = base + "_" + identifier + handle
# #     path = csv_dir + '/' + fn
# #     return path

# def create_dir_if_not_exists(dir_path, warn=True):
#     if not os.path.exists(dir_path):
#         os.makedirs(dir_path)
#         warnings.warn("Created new directory at " + str(base_dir))
#         return True
#     return False

# def save_path_generator(corpus_dir, file_purpose_dir, file_name, file_type, identifier=None, part=None):
#     proj_dir = '/nfs/proj-repo/AAARG-dissertation'
#     base_dir = proj_dir + '/' + 'dataset'  # base folder for storing corpus files
#     create_dir_if_not_exists(base_dir)
#     # select appropriate corpus directory (e.g. trects-filtered-2014)
#     path = base_dir + '/' + corpus_dir
#     create_dir_if_not_exists(path)
#     # ensure is from pre-selected file_purposes
#     file_purposes = ["corpus", "nuggets", "topics", "embed_labels", "updates"]
#     if file_purpose_dir not in file_purposes:
#         raise ValueError("File purpose must be in defined file purposes")
#     path += '/' + file_purpose_dir
#     create_dir_if_not_exists(path)
    
#     # create file name
#     path += '/' + file_name
#     if identifier is not None:
#         path += '_' + identifier
#     if part is not None:
#         path += '_' + part
#     path += file_type
    
#     return path
        

## Markup Loading Functions

In [7]:
# open and get beautifulsoup object from markup file
def open_markup_file(addr, gz=True, xml=False, verbose=False):
    markup = None
    f = None
    
    if verbose:
        print(addr)

    if gz:
        f = gzip.open(addr)
        if verbose:
            print("gz file opened")
    else:
        f = open(addr)
        
    if xml == False:
        markup = bs(f, "lxml")  # using lxml parser for speed
    else:
        markup = bs(f, "xml")
        
    f.close()
    return markup


# parse markup and return 2D list [entry:tags]
def parse_markup(markup, entry_list, tag_list, find_tag, topic_id=None):
    for e in markup.find_all(find_tag):
        entry = OrderedDict.fromkeys(tag_list)
        if topic_id is not None:
            entry['topic_id'] = topic_id
        for c in e.children:  # children use direct children, descendants uses all
            if str(c.name).lower() in entry:
                entry[c.name] = str(c.string)
            elif c.name is None and c.string != '\n':  # inner body of <doc> tag
                entry['text'] = str(c.string)
        entry_list.append(list(entry.values()))
        
            
# recursively find gz html files from a directory address
def search_dir(path):    
    # separate the subdirectories and html files 
    # (help maintain sequential order of insertion)
    gz_paths = []
    for f in os.scandir(path):
        if os.path.splitext(f.path)[-1].lower() == ".gz":
            gz_paths.append(f.path)
    
    return gz_paths


def list_to_dataframe(markup_list, tags):
    return pd.DataFrame(markup_list, columns=tags)

## Dataframe Loading

### Control Functions

In [237]:
def topic_id_as_int(topic_id):
    try:
        topic_id = int(topic_id)
        return topic_id
    except ValueError:  # non-standard topic_id, e.g. 'TS14.18'
        split = topic_id.split(".", 2)  
        try:
            match = split[0].upper()
            if match == "TS14" or match == "14":
                topic_id = int(split[1])  # extract int '18'
                return topic_id
            else:
                raise ValueError()
        except ValueError:
            return None  # no discernable topic_id

def convert_df_topic_id(df, col_name="query_id", remove_null=True, in_place=False):
    conv_df = df
#     print("df[" + col_name + "] unique entries: " + conv_df[col_name].unique())
    conv_df[col_name] = conv_df[col_name].apply(lambda x: topic_id_as_int(x))
    if remove_null:
        conv_df = conv_df[conv_df[col_name].notna()]
        conv_df = conv_df.astype({col_name:int})
    if in_place:
        df = conv_df
        return df
    else:
        return conv_df

def find_duplicates(df):
    seen = set()
    seen_twice = set()
    for docid in df['docid']:
        if docid not in seen:
            seen.add(docid)
        else:
            seen_twice.add(docid)
    return seen_twice

def load_df_control(save_path, load_func, save=True, force_reload=False, compression='gzip', 
                    name=None, verbose=True, path_handler=None):
    df = None
    save_path = convert_to_list(save_path)  # allows loading groups of saved files same way as singular paths
    if name is not None and verbose:
        print("Loading " + name)
    if not file_exists(save_path) or force_reload:
        if len(save_path) > 1:
            raise ValueError("There should only be one path to save to if no save paths already exist")
        df = load_func()
        if verbose:
            print("df loaded")
        if save:
            # in case of loading df from original file, should only be one save_path
            df.to_csv(save_path[0], compression=compression)
            if path_handler is not None:
                path_handler.update_path_exists(save_path[0])
            if verbose:
                print("saved at: " + str(save_path[0]))
    else:
        df_list = []
        for path in save_path:
            df_list.append(pd.read_csv(path, compression=compression))
        if len(df_list) > 1:
            df = pd.concat(df_list)
        else:
            df = df_list[0]
        if verbose:
            print("loaded from file")
    for col in df.columns:  # bug loading from certain files produces nonsense columns
        if "Unnamed" in col:
            del df[col]
    if verbose:
        print(display(df[0:4]))
    return df

### Dataframes from Corpus Files

#### Topics

In [278]:
# load topics into dataframe
def __load_topics(path, verbose=True):
    topics_list = []
    path = convert_to_list(path)
    for p in path:
        parse_markup(open_markup_file(p, gz=False, xml=True), 
                        topics_list, topic_tags, "event")
    df = list_to_dataframe(topics_list, topic_tags)
    
    df = convert_df_topic_id(df, col_name='id', remove_null=True)
    # drop any duplicates found over the files
    prev_size = len(df)
    df = df.drop_duplicates(subset=['id'], keep='first')  # no duplicate documents
    if verbose:
        num_removed = len(df) - prev_size
        print(str(num_removed) + " duplicate documents removed from topics df")
    return df

def load_topics(saved_path, load_path=None, save=True, force_reload=False, verbose=True, path_handler=None):
    topics = load_df_control(saved_path, lambda: __load_topics(load_path, verbose=verbose), 
                             save=save, force_reload=force_reload, name="topics", verbose=verbose, path_handler=path_handler)
    return topics

# topics = load_topics()

#### Main Corpus Files

In [212]:
# load all formatted gzipped html files into dataframe

def __load_corpus(corpus_dir, doc_tags=None, topic_ids=None, split_every=None, split_start_doc=None,
                 drop_duplicates=True, verbose=True):
    if doc_tags is None:
        doc_tags = ['topic_id','streamid', 'docid', 'yyyymmddhh', 'kbastream', 'zulu', 'epoch', 'title', 'text', 'url'] # doc fields
    df = pd.DataFrame(columns=doc_tags)
    
    for topic_id in topic_ids:
        print("Loading topic " + str(topic_id) + "...")
        topic_list = []
        topic_path = corpus_dir + '/' + str(topic_id)
        gz_paths = search_dir(topic_path)
        
        if split_every is not None and split_start_doc is not None:
            end_split = split_start_doc + split_every
            if end_split >= len(gz_paths):  # last section
                end_split = len(gz_paths) - 1
            gz_paths = gz_paths[split_start_doc:end_split]
        
        for gz_path in tqdm(gz_paths, position=0, leave=True):
            parse_markup(open_markup_file(gz_path, verbose=False),
                             topic_list, doc_tags, "doc", topic_id=topic_id)
        topic_df = list_to_dataframe(topic_list, doc_tags)
        df = df.append(topic_df)
    df['epoch'] = pd.to_numeric(df['epoch'])
    if drop_duplicates:
        prev_size = len(df)
        df = df.drop_duplicates(subset=['streamid'], keep='first')  # no duplicate documents
        if verbose:
            num_removed = len(df) - prev_size
            print(str(num_removed) + " duplicate documents removed from corpus")
    return df

def load_corpus(save_path, corpus_dir=None, doc_tags=None, topic_ids=None, split_every=None, split_start_doc=None,
                save=True, force_reload=False, verbose=True, path_handler=None, drop_duplicates=True,
               compression='gzip'):
    
    corpus = load_df_control(save_path, 
                             lambda: __load_corpus(corpus_dir, doc_tags=doc_tags, 
                                                   topic_ids=topic_ids, split_every=split_every,
                                                   split_start_doc=split_start_doc, 
                                                   drop_duplicates=drop_duplicates, verbose=verbose), 
                             save=save, force_reload=force_reload, name="corpus", verbose=verbose, path_handler=path_handler)
    # remove duplicate documents from corpus and save if required
    if drop_duplicates:
        if corpus['streamid'].duplicated().any():  # if there are any duplicates
            prev_size = len(corpus)
            corpus = corpus.drop_duplicates(subset=['streamid'], keep='first')  # get rid of them
#             if save:
#                 save_path = convert_to_list(save_path)
#                 for path in save_path:
#                     df.to_csv(path, compression=compression)
            if verbose:
                num_removed = len(corpus) - prev_size
                print(str(num_removed) + " duplicate documents removed from corpus")
    
    if verbose:
        print("Corpus loaded succesfully: " + str(len(corpus)) + " documents loaded.")
    return corpus

# corpus = load_corpus(doc_tags=doc_tags)

#### Nuggets (Evaluation Technique)

In [104]:
# import re

# def create_nugget_df(corpus_df, topic_ids=None, nuggets_tsv=None, nuggets_tsv_path=None, spacy_model=None):
#     """Dataframe containing nugget data and its appearances in corpus"""
#     def create_entry(row, reg_cols, multi_col_vals=None):
#         entry_dict = {}
#         for col in reg_cols:
#             entry_dict[col] = row[col]
#         if multi_cols is not None:
#             for k,v in multi_col_vals.items():
#                 entry_dict[k] = v
#         return entry_dict
    
#     def find_nugget_in_text(text, nugget_text):
#         """Retrieve the sentence that the nugget appears in the text
#         Only when spacy model is not None
#         """
#         text = nlp(text)
#         sentences = list(text.sents)
#         sentences = [s.text for s in sentences if len(s) != 0]
# #         target_sent = None
#         target_sents = []
#         for sent in sentences:
#             if nugget_text in sent:
# #                 target_sent = sent
# #                 break
#                 target_sents.append(sent)
#         print("nugget_text: " + str(nugget_text))
#         print("target_sents: " + str(target_sents))
# #         if target_sent is None:
#         if len(target_sents) == 0:
#             raise Exception("Nugget not found in sentences but identified by df.str.contains")
#         #return target_sent
#         return target_sents
    
#     # the data we are going to store in our processed nugget_df
#     entry_list = []
#     reg_cols = ['query_id', 'nugget_id', 'importance', 'nugget_len', 'nugget_text']
#     multi_cols = ['docid', 'streamid', 'epoch', 'yyyymmddhh']  # columns with multiple entries
#     num_cols = ['query_id', 'importance', 'nugget_len', 'epoch']  # comvert these columns to numerical vals
    
#     if nuggets_tsv is None:  # load nuggets_tsv in function instead
#         nuggets_tsv = pd.read_csv(nuggets_tsv_path, "\t")
       
#     # convert topic_ids to int standard
#     nug_tsv = convert_df_topic_id(nuggets_tsv, col_name='query_id', remove_null=True)
    
#     # target only selected topic_ids if not None
#     if topic_ids is not None:
#         nug_tsv[nug_tsv['query_id'].isin(topic_ids)]
    
#     pbar = tqdm(total=len(nug_tsv), position=0, leave=True)
#     for index, row in nug_tsv.iterrows():
#         # find where nugget appears in text
#         nug_text = row['nugget_text']
#         topic_id = 0
#         try:
#             topic_id = int(row['query_id'])  # make sure pattern match in correct topic
#         except ValueError:
#             pbar.update()
#             continue  # topic_id is unknown string in tsv file, e.g. "TS13.07"
#         appears = corpus_df[corpus_df['topic_id'] == topic_id]
#         appears = appears[appears['text'].str.contains(re.escape(nug_text))]  # make sure no accidental regex pattern
        
#         # gather information on docs it appears in
#         dups = find_duplicates(appears)  # get docids where nugget appears
#         for docid in dups:
#             upd = appears[appears['docid'] == docid]  # get docs with this docid
#             for i, r in upd.iterrows():  # gather info on each doc with this docid (e.g. streamid, epoch etc.)
#                 multi_col_vals = {}
#                 for multi_col in multi_cols:
#                     multi_col_vals[multi_col] = r[multi_col]
                    
#                 entry = create_entry(row, reg_cols, multi_col_vals=multi_col_vals)
                
#                 if spacy_model is not None:  # find sentence in document using spacy tokenized sentences
#                     found_sent = find_nugget_in_text(r['text'], nug_text)
#                     entry['sentence_in_doc'] = found_sent
                
#                 entry_list.append(entry)
#         pbar.update()
#     pbar.close()
    
#     print("len entry_list: " + str(len(entry_list)))
    
#     # form multi-index nugget dataframe
# #     reg_cols.extend(multi_cols)  # get new multiindex order
#     nugget_df = pd.DataFrame(entry_list)
#     nugget_df[num_cols] = nugget_df[num_cols].apply(pd.to_numeric, errors='coerce', axis=1)  # convert appropriate cols to numerical values
#     nugget_df.rename(columns={'query_id':'topic_id'}, inplace=True)  # topic_id matches other dataframes
#     return nugget_df

# def load_nugget_df(save_path, corpus_df, topic_ids=None, spacy_model=None, nuggets_tsv=None, nuggets_tsv_path=None, 
#                    path_handler=None, save=True, force_reload=False, verbose=True):
#     nugget_df = load_df_control(save_path, lambda: create_nugget_df(corpus_df, nuggets_tsv=nuggets_tsv,
#                                                                    nuggets_tsv_path=nuggets_tsv_path,
#                                                                    spacy_model=spacy_model, topic_ids=topic_ids), 
#                                 save=save, force_reload=force_reload, name="nugget_df", verbose=verbose,
#                                path_handler=path_handler)
#     return nugget_df


In [328]:
def create_nugget_df(corpus_df, nuggets_tsv=None, matches_tsv=None, nuggets_tsv_path=None,
                     matches_tsv_path=None, topic_ids=None, spacy_if_not_found=True, verbose=True):
    def check_load_tsv(tsv, path):
        if tsv is None:
            if path is None:
                raise Exception("Must either pass the tsv file or the path to load it")
            else:  # load tsv
                path = convert_to_list(path)
                tsv = []
                for p in path:
                    tsv.append(pd.read_csv(p, "\t"))
                tsv = pd.concat(tsv, ignore_index=True)
        return tsv
    
    # perform check if tsvs or their paths have been passed
    nuggets_tsv = check_load_tsv(nuggets_tsv, nuggets_tsv_path)
    matches_tsv = check_load_tsv(matches_tsv, matches_tsv_path)
    
    def parse_update_id(update_id):
        """Separate update_id into component streamid and sent_id"""
        update_id = update_id.split("-")
        sent_id = int(update_id[-1])
        streamid = "-".join(update_id[:-1])
        return streamid, sent_id
    
    def find_nugget_spacy(text, match_start, nlp):
        if nlp is None:
            nlp = spacy.load("en_core_web_sm")
        spacy_text = nlp(text)
        spacy_sents = list(nlp(text).sents)
        nug = None
        char_count = 0
        for s in spacy_sents:  # find sentence by where characters start
            s = str(s)  # convert from spacy tokens to string
            s_len = len(s)
            if char_count + s_len > match_start:
                nug = s
                break
            char_count += s_len
        return nug
    
    def find_nugget_in_text(text, sent_id, match_start, nlp):
        """Retrieve sentence at index sent_id"""
        split = text.splitlines()
        if split[0] == "":
            sent_id += 1  # first entry is empty, adjust offset
        nug = None
        try:
            nug = split[sent_id]
        except IndexError:  # increment has pushed offset out of bounds
            try:
                sent_id -= 1
                nug = split[sent_id]
            except IndexError as e:  # sent_id does not match text indexing
                if spacy_if_not_found:
                    nug = find_nugget_spacy(text, match_start, nlp)
                    sent_id = None  # set to a null value to differentiate
        return nug, sent_id
    
    nlp = None  # spacy model, load if needed
    entry_list = []  # list of dicts to build dataframe
    
    # what columns from each dataframe to extract to put into nugget_df
    nug_tsv_cols = ['importance', 'nugget_len', 'nugget_text']
    mat_tsv_cols = ['query_id', 'match_start', 'match_end']
    corp_cols = ['docid', 'streamid', 'epoch']
    # reference what columns to convert from string into numerical values
    num_cols = ['query_id', 'importance', 'nugget_len', 'epoch', 'sent_id', 'match_start', 'match_end']
    
    # convert topic_ids to int standard
    # set to new var to allow passing same unchanged nuggets/matches_tsv each time
    nug_tsv = convert_df_topic_id(nuggets_tsv, col_name='query_id', remove_null=True)
    mat_tsv = convert_df_topic_id(matches_tsv, col_name='query_id', remove_null=True)
    
    # target only selected topic_ids if not None
    if topic_ids is not None:
        for topic_id in topic_ids:
            nug_tsv = nug_tsv[nug_tsv['query_id'] == topic_id]
            mat_tsv = mat_tsv[mat_tsv['query_id'] == topic_id]
    
    missed_streamids = []  # store streamids not found for debug purposes
    missed_nuggetids = []  # debug purposes
    missed_sentid_streamids = []  # streamid where sent_id indexing out of bounds
    pbar = tqdm(total=len(mat_tsv), position=0, leave=True)
    for index, row  in mat_tsv.iterrows():
        entry = {}
        
        # get streamid and sentid of nugget occurence
        streamid, sent_id = parse_update_id(row['update_id'])
        
        # find occurence in corpus
        occur = corpus_df[corpus_df['streamid'] == streamid]
        if len(occur) == 0:
            missed_streamids.append(streamid)
            continue
        elif len(occur) > 1:
            if verbose:
                print("Number of entries with streamid: " + str(len(occur)))
                print(display(occur))
            raise Exception("There should be one entry in corpus with given streamid " + str(streamid))
        occur = occur.iloc[0].to_dict()
        
        # get text of the occurence
        occur_text = occur['text']
        match_start = int(row['match_start'])
        
        # get text of the nugget
        nug_row = nug_tsv[nug_tsv['nugget_id'] == row['nugget_id']]
        if len(nug_row) != 1:
            missed_nuggetids.append(row['nugget_id'])
            continue
        nug_row = nug_row.iloc[0].to_dict()
        
        # add columns from each dataframe
        for col in mat_tsv_cols:
            entry[col] = mat_tsv.at[index, col]
        for col in nug_tsv_cols:
            entry[col] = nug_row[col]
        # adding these columns here to control order of columns in final df
        found_sent, sent_id = find_nugget_in_text(occur_text, sent_id, match_start, nlp)
        entry['sent_in_text'] = found_sent
        entry['sent_id'] = sent_id
        for col in corp_cols:
            entry[col] = occur[col]
        
        if sent_id is None or found_sent is None:  # sent_id indexing was wrong
            missed_sentid_streamids.append(streamid)
        
        entry_list.append(entry)
        
    if verbose:
        print("Nugget entries were generated for " + str(len(entry_list)) + " nuggets. There were "
             + str(len(missed_streamids)) + " found in matches.tsv but not in corpus")
        print("There were " + str(len(missed_nuggetids)) + " nugget_ids found in matches.tsv but not in nuggets.tsv")
        print(str(len(missed_sentid_streamids)) + " out of " + str(len(entry_list)) + 
              " streamids had out of bounds sent_ids")
        
    nugget_df = pd.DataFrame(entry_list)
    if len(nugget_df) > 0:
        nugget_df[num_cols] = nugget_df[num_cols].apply(pd.to_numeric, errors='coerce', axis=1)  # convert appropriate cols to numerical values
        nugget_df.rename(columns={'query_id':'topic_id'}, inplace=True)  # topic_id matches other dataframes
    
    if verbose:
        print("nugget_df entries: " + str(len(nugget_df)))
    
    return nugget_df

In [313]:
def load_nugget_df(save_path, corpus_df, topic_ids=None,path_handler=None, save=True, force_reload=False, 
                   verbose=True, nuggets_tsv=None, matches_tsv=None, nuggets_tsv_path=None, matches_tsv_path=None,
                  spacy_if_not_found=True):
    
    nugget_df = load_df_control(save_path, 
                    lambda: create_nugget_df(corpus_df, nuggets_tsv=nuggets_tsv, verbose=verbose,
                                             matches_tsv=matches_tsv, nuggets_tsv_path=nuggets_tsv_path, 
                                             matches_tsv_path=matches_tsv_path, topic_ids=topic_ids,
                                            spacy_if_not_found=spacy_if_not_found), 
                                save=save, force_reload=force_reload, name="nugget_df", verbose=verbose,
                               path_handler=path_handler)
    return nugget_df

In [103]:
nugget_df = load_nugget_df("/nfs/proj-repo/AAARG-dissertation/l_d/nugget_df.csv.gz")
print(display(nugget_df[0:5]))

Loading nugget_df
loaded from file


,Unnamed: 0,topic_id,nugget_id,importance,nugget_len,nugget_text,docid,streamid,epoch,yyyymmddhh
0,0,1,VMTS13.01.052,3,2,Hundreds injured,dd95d5dbbff443c3ddae4e34a5d2e9c1,1330041420-dd95d5dbbff443c3ddae4e34a5d2e9c1,1330041420,2012-02-23-23
1,1,1,VMTS13.01.052,3,2,Hundreds injured,dd95d5dbbff443c3ddae4e34a5d2e9c1,1330041420-dd95d5dbbff443c3ddae4e34a5d2e9c1,1330041420,2012-02-23-23
2,2,1,VMTS13.01.054,1,3,"February 22, 2012",f66f6668504592a391345e012800469c,1329944400-f66f6668504592a391345e012800469c,1329944400,2012-02-22-21
3,3,1,VMTS13.01.054,1,3,"February 22, 2012",f66f6668504592a391345e012800469c,1329944400-f66f6668504592a391345e012800469c,1329944400,2012-02-22-21


None


,Unnamed: 0,topic_id,nugget_id,importance,nugget_len,nugget_text,docid,streamid,epoch,yyyymmddhh
0,0,1,VMTS13.01.052,3,2,Hundreds injured,dd95d5dbbff443c3ddae4e34a5d2e9c1,1330041420-dd95d5dbbff443c3ddae4e34a5d2e9c1,1330041420,2012-02-23-23
1,1,1,VMTS13.01.052,3,2,Hundreds injured,dd95d5dbbff443c3ddae4e34a5d2e9c1,1330041420-dd95d5dbbff443c3ddae4e34a5d2e9c1,1330041420,2012-02-23-23
2,2,1,VMTS13.01.054,1,3,"February 22, 2012",f66f6668504592a391345e012800469c,1329944400-f66f6668504592a391345e012800469c,1329944400,2012-02-22-21
3,3,1,VMTS13.01.054,1,3,"February 22, 2012",f66f6668504592a391345e012800469c,1329944400-f66f6668504592a391345e012800469c,1329944400,2012-02-22-21
4,4,1,VMTS13.01.054,1,3,"February 22, 2012",ddd856e0a350c52b7c078c9bcdd609d9,1329930660-ddd856e0a350c52b7c078c9bcdd609d9,1329930660,2012-02-22-17


None


#### Update Dataframe (Temporal Information)

In [11]:
def create_update_df():
    """Data Frame containing information about docs which have updates/multiple instances in corpus"""
    def create_entry(row, col_tags):
        entry = {}
        for col in col_tags:
            entry[col] = row[col]
        return entry
    
    col_tags = ['docid', 'streamid', 'epoch', 'yyyymmddhh', 'zulu']
    entry_list = []
    dups = find_duplicates(corpus)
    for docid in tqdm(dups, position=0, leave=True):
        d = corpus[corpus['docid'] == docid]
        for index, row in d.iterrows():
            entry = create_entry(row, col_tags)
            entry_list.append(entry)
             
    update_df = pd.DataFrame(entry_list)
    update_df = update_df.set_index(col_tags)
    return update_df

def load_update_df(save=True, force_reload=False, verbose=True):
    update_df = load_df_control(update_csv_path, create_update_df, 
                                save=save, force_reload=force_reload, name="update_df", verbose=verbose)
    return update_df

# update_df = load_update_df()

## Supervised Learning Input Data (Embeddings) / Labels (Model Summary)

In [12]:
# preprocess corpus into cleaned sentences
# create sentence embeddings of corpus text
# create embeddings from where nuggets appear in article
# match them together in df

In [12]:
class SupervisedTrainingGenerator:
    """Currently not matching nuggets to correct sentence_id"""
    def __init__(self, spacy_model_selector="en_core_web_sm"):
        self.sent_model = self.init_sent_model()
        self.spacy_model_selector = spacy_model_selector
        self.nlp = None
        
    def generate(self, corpus_df, nugget_df, topic_ids=None, save=True, force_reload=False, verbose=True):
        if topic_ids is not None:
            self.supervised_df = {}
            for topic_id in topic_ids:
                print("Processing topic " + str(topic_id))
                t_corpus = corpus_df[corpus_df['topic_id'] == topic_id]
                t_nugget = nugget_df[nugget_df['topic_id'] == topic_id]
                t_path = supervised_path_generator("topic" + str(topic_id))
                self.supervised_df[topic_id] = load_df_control(t_path, 
                                                lambda: self.__generate(t_corpus, t_nugget), 
                                                save=save, force_reload=force_reload, 
                                                verbose=verbose, name="supervised_df" + str(topic_id))
                self.sent_model = self.init_sent_model()
        else:
            # maybe check here load different csv with different topic_ids
            self.supervised_df = load_df_control(supervised_csv_path, 
                                                lambda: self.__generate(corpus_df, nugget_df), 
                                                save=save, force_reload=force_reload, 
                                                verbose=verbose, name="supervised_df")
        return self.supervised_df
    
    def __generate(self, corpus_df, nugget_df):
        # create df each row being a sentence, it's embedding, sent_id, is_nugget, nugget_text, topic_id, streamid, docid etc.
        supervised = []
        for index, article in tqdm_notebook(corpus_df.iterrows(), total=corpus_df.shape[0], position=0, leave=True):
            # preprocess sentences
            sentences = self.preprocess_text(article['text'])
            sent_ids, sentences, embeddings = self.sent_embeddings(sentences)
            
            # if nuggets in article, get the index of the sentence
            streamid = article['streamid']
            article_nugs = self.nugget_matching_sent(streamid, nugget_df, sentences)
            
            # create dictionary for later creating dataframe
            for sent_id, sent, emb in zip(sent_ids, sentences, embeddings):
                t_id = article['topic_id']
                epoch = article['epoch']
                is_nugget = False
                nugget_text = None
                nugget_id = None
                
                nug_dict_index = None
                # check if nugget
                try:
                    nug_dict_index = article_nugs['sent_id'].index(sent_id)  # throws if not in list
                    is_nugget = True
                    nugget_text = article_nugs['nugget_text'][nug_dict_index]
                    nugget_id = article_nugs['nugget_id'][nug_dict_index]
                except ValueError:
                    pass # current sentence is not nugget
                
                s_dict = {"topic_id":t_id, "streamid":streamid, "epoch":epoch, "sent_id":sent_id, 
                          "sentence":sent, "embedding":emb, "is_nugget":is_nugget, 
                          "nugget_id":nugget_id, "nugget_text":nugget_text}
                supervised.append(s_dict)
                
        supervised_df = pd.DataFrame(supervised)
        return supervised_df
                
                
    def nugget_matching_sent(self, streamid, nugget_df, sentences):
        # find sent index of where nugget appears in text
        matches = {"sent_id":[], "nugget_text":[], "nugget_id":[]}
        for index, nug in self.nuggets_in_article(streamid, nugget_df).iterrows():
            match = None
            nug_text = nug['nugget_text']
            for i in range(len(sentences)):
                match = None
                if sentences[i] in nug_text:
                    match = i  # only take first appearance in article if multiple exist
                    break
            if match is not None:
                matches["sent_id"].append(match)
                matches["nugget_text"].append(nug_text)
                matches['nugget_id'].append(nug['nugget_id'])
        return matches
        
                
    def nuggets_in_article(self, streamid, nugget_df):
        # find streamid in nugget_df
        nug_rows = nugget_df[nugget_df['streamid'] == streamid]
        return nug_rows
                
            
    def preprocess_text(self, text, use_spacy=False):
        # remove first char if \n
#         if text[:1] == "\n":
#             text = text[1:]
        
        sentences = None
        if spacy:
            if self.nlp is None:
                self.nlp = spacy.load(self.spacy_model_selector)
            text = self.nlp(text)
            sentences = list(text.sents)
            sentences = [s.text for s in sentences if len(s) != 0]
        else:  # split by newline
            sentences = text.splitlines()
        return sentences
        
    def sent_embeddings(self, sentences):
        # use sentence-transformers embeddings
        result = self.sent_model.encode(sentences, show_progress_bar=False)
        sent_ids = []
        tokens = []  # sentences as text
        embeddings = []
        for i, (tok, emb) in enumerate(zip(sentences,result)):
            sent_ids.append(i)
            tokens.append(tok)
            embeddings.append(emb)
#         embeddings = np.stack(embeddings)
        return sent_ids, tokens, embeddings
#         # we normalize embeddings, so that euclidian distance is equivalent to cosine distance
#         self.normed_embeddings = (embeddings.T / (embeddings**2).sum(axis=1) ** 0.5).T

    def init_sent_model(self):
        sent_model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
        return sent_model

In [14]:
# super_gen = SupervisedTrainingGenerator()
# super_df = super_gen.generate(corpus, nugget_df)

In [15]:
# print(display_df[0:5])

In [16]:
# sli = pd.DataFrame.copy(super_gen.supervised_df[1][0:5], deep=True)
# # testees = sli['nugget_text']
# # testees[2] = "poop"
# # sli['nugget_text'] = testees
# # print(sli['nugget_text'].unique())
# e_df_test = pd.DataFrame(columns=sli.columns)
# print(list(e_df_test[e_df_test['epoch']==None]['topic_id']))

## Filter the Larger Trects Dataset

In [13]:
class TrectsFilter:
    def __init__(self):
        self.base_dir = '/nfs/trects-kba2014'
        self.updates_dir = "/nfs/TemporalSummarization/ts14/results"
        self.updates_csv_paths = self.generate_update_paths()
        self.save_dir = '/nfs/mine-trects-kba2014-filtered'
        self.proc_history_path = self.save_dir + '/' + 'process_history.pickle'
        self.proc_history = None
        self.streamids = set()

    def generate_update_paths(self, verbose=True):
        up_dir = "/nfs/TemporalSummarization"
        ts_dirs = ["ts13", "ts14", "ts15"]
        target_files = ['updates_sampled.extended.tsv', 'updates_sampled.tsv',
                                 'matches.tsv']
        
        up_paths = []
        wrong_paths = []
        for ts_dir in ts_dirs:
            for target_file in target_files:
                full_path = up_dir + '/' + ts_dir + '/results/' + target_file
                if os.path.exists(full_path):
                    up_paths.append(full_path)
                else:
                    wrong_paths.append(full_path)
        if verbose:
            print("Attempted to find streamids in these files, but no path exists:")
            print(wrong_paths)
            print("")
        return up_paths
        
        
    def create_filtered_dataset(self, force_reload=False, verbose=True, no_soup=True):
        """ Outline of Process
        1. Find streamids
            1.1 open updates_sampled.tsv file (or updates_sampled.extended.tsv)
            1.2 Scrape update_id column
            1.3 transform into streamid (drop last hyphenated numbers (these are sentenceids))
            1.4 Put streamids into datastructure for comparing (e.g. set)
        2. Create a new directory for each topic folder there is in target dir
        3. Opening up documents
            3.1 Go for each topic folder
            3.2 Open up each document
            3.3 Parse into html tree
            3.4 if streamid matches, store locally in memory buffer
            3.5 when buffer is size of however many docs are in other html.gz files, print those docs to a new file
            3.6 name this file something like a number, not dates like other files, save with same file extension/format
        4. Quick test
            4.1 Open up a topicid using load corpus
        5. Final check
            5.1 Add in final check that will only run this if it hasn't already been generated
        """
        # get streamids for docs that we will filter for
        for update_csv_path in self.updates_csv_paths:
            self.get_streamids(update_csv_path)
        if verbose:
            print("Number of streamids searching for: " + str(len(self.streamids)))

        # get topicids from folder names
        topic_ids = [int(tid) for tid in os.listdir(self.base_dir) if tid.isdigit()]
        topic_ids.sort()
        
        # create dir to save filtered corpus to
        self.create_dir(self.save_dir)
        
        # load history of files already processed if exists
        self.load_process_history_dict(topic_ids)
        
        
        for topic_id in tqdm_notebook(topic_ids, position=0, leave=True):
            # create save directory
            topic_save_dir = self.save_dir + '/' + str(topic_id)
            self.create_dir(topic_save_dir)
            
            # get paths for files in target topic dir
            topic_dir = self.base_dir + '/' + str(topic_id)
            gz_paths = search_dir(topic_dir)
            
            # remove already processed files
            if not force_reload:
                if verbose:
                    prev_printed = [x for x in gz_paths if x in self.proc_history[topic_id]]
                    print("Previously processed " + str(len(prev_printed)) + " of " + str(len(gz_paths))
                         + " paths for topic " + str(topic_id))
                gz_paths = [x for x in gz_paths if x not in self.proc_history[topic_id]]
            
            if verbose:
                if len(gz_paths) > 0:
                    print("Processing topic " + str(topic_id))
            
            # process each file
            for gz_path in tqdm_notebook(gz_paths):
                if no_soup == True:
                    matches = self.process_file(gz_path, verbose=verbose)
                    if verbose:
                        print("len matches: " + str(len(matches)))
                    save_path = self.get_file_save_path(topic_id, gz_path)
                    self.write_docs_to_file(matches, save_path, no_soup=True, verbose=verbose)
                else:
                    # get file markup
                    markup = open_markup_file(gz_path, verbose=verbose)  # 50MB file proving hard for beautifulsoup
                    # get docs in file that are in streamids
                    matches = self.retrieve_matching_docs(markup, verbose=verbose)
                    save_path = self.get_file_save_path(topic_id, gz_path)
                    # write file and save results
                    self.write_docs_to_file(matches, save_path, verbose=verbose)
                self.proc_history[topic_id].add(gz_path)
                self.save_process_history_dict(verbose=verbose)
                
        print("Finished filtering corpus")
        
    def process_file(self, filepath, verbose=True):
        if verbose:
            print("Processing file at: " + str(filepath))
        matching_docs = []
        doc_buffer = []
        found_sid = False
        save_doc = False
        with gzip.open(filepath, 'rt') as f:
            for line in f:
                doc_buffer.append(line)  # add cur line to buffer
                buff_len = len(doc_buffer)
                if buff_len > 1:
                    if not found_sid:
                        if self.is_tag(line, tag="streamid"):
                            found_sid = True
                            sid = self.get_inner_tag(line, tag="streamid", remove_whitespace=True)
                            if sid in self.streamids:
                                save_doc = True
                    if self.is_tag(line, tag="doc", start_tag=False, end_tag=True):
                        if save_doc:  # if has matching streamid save doc file
                            matching_docs.append(doc_buffer)
                        doc_buffer = []
                        save_doc = False
                        found_sid = False
                        
                elif self.is_tag(line, tag="doc", start_tag=True, end_tag=False):
                    # reset variables
                    doc_buffer = []
                    doc_buffer.append(line)
        return matching_docs
                
                
    def is_tag(self, line, tag="streamid", start_tag=True, end_tag=True):
        start, end = self.create_tags(tag)
        start_true = False
        end_true = False
        
        if re.search(start, line, re.IGNORECASE):
            start_true = True
        if re.search(end, line, re.IGNORECASE):
            end_true = True
        
        if start_tag == True:
            if end_tag == True:
                return start_true and end_true
            else:
                return start_true
        else:
            return end_true
        
    def create_tags(self, tag):
        start_tag = "<" + tag + ">"
        end_tag = "</" + tag + ">"
        return start_tag, end_tag
            
    def get_inner_tag(self, line, tag="streamid", remove_whitespace=False):
        start_tag, end_tag = self.create_tags(tag)
        no_tags = line.replace(start_tag,'').replace(end_tag,'')
        no_tags = line.replace(start_tag.upper(), '').replace(end_tag.upper(),'')  # repeat for uppercase
        if remove_whitespace == True:
            no_tags = no_tags.rstrip()
        return no_tags
        
    def save_process_history_dict(self, verbose=True):
        with open(self.proc_history_path, 'wb') as handle:
            pickle.dump(self.proc_history, handle, protocol=pickle.HIGHEST_PROTOCOL)
            if verbose:
                print("saved proc_history")
        
    def load_process_history_dict(self, topic_ids):
        if os.path.exists(self.proc_history_path):
            with open(self.proc_history_path, 'rb') as handle:
                self.proc_history = pickle.load(handle)
            return True
        else:
            self.proc_history = self.create_process_history_dict(topic_ids)
            return False
        
    def create_process_history_dict(self, topic_ids):
        """Create a dictionary to keep track of what files have already been searched"""
        proc_history = {}
        for topic_id in topic_ids:
            proc_history[int(topic_id)] = set()  # sets have faster indexing
        return proc_history

                
    def get_file_save_path(self, topic_id, gz_path):
        filename = self.get_filename_from_gz_path(gz_path)
        save_path = self.save_dir + '/' + str(topic_id) + '/' + filename
        return save_path
        
                
    def get_filename_from_gz_path(self, gz_path):
        split = gz_path.split("/")
        filename = split[-1]
        return filename  # return with file extension on
                

    def write_docs_to_file(self, doc_list, save_path, no_soup=False, verbose=True):
        # transform docs into string
        if len(doc_list) > 0:  # don't write empty files
            out = ""
            if no_soup == True:
                out += "<html>\n"
                for doc in doc_list:
                    out += "".join(list(map(str, doc)))
                    out += "\n"
                out += "</html>"
            else:
                out = "\n".join(list(map(str, doc_list)))
            # write
            with gzip.open(save_path, "wt") as f:
                f.write(out)
                if verbose:
                    print("File written to: " + str(save_path))
        
            
    def retrieve_matching_docs(self, markup, verbose=False):
        """Retrieve docs with matching streamids from markup"""
        matches = []
        doc_count = 0
        match_count = 0
        for doc in markup.find_all("doc"):
            d_streamid = str(doc.find("streamid").string)
            if d_streamid in self.streamids:  # matching doc
                matches.append(doc)
                match_count += 1
            doc_count +=1
        if verbose:
            print("doc count: " + str(doc_count) + "\nmatch_count: " + str(match_count))
        return matches
    
    def get_streamids(self, path):
        # read tsv file
        updates_csv = pd.read_csv(path, "\t")
        # take column with streamids
        updateids = list(updates_csv['update_id'])
        for updateid in updateids:
            streamid = self.parse_streamid(updateid)
            self.streamids.add(streamid)
        return self.streamids
        
    def parse_streamid(self, updateid):
        """Convert updateid in format: epoch-docid-sentid into epoch-docid"""
        split = updateid.split("-")
        split = split[:-1]  # remove sentid from end
        streamid = "-".join(split)
        return streamid
    
    def create_dir(self, dir_path):
        if not os.path.exists(dir_path):
            os.makedirs(dir_path)
            print("Created new directory at " + str(dir_path))

In [129]:
trectsfilter = TrectsFilter()
trectsfilter.create_filtered_dataset(verbose=True, force_reload=False, no_soup=True)

Attempted to find streamids in these files, but no path exists:
['/nfs/TemporalSummarization/ts13/results/updates_sampled.extended.tsv', '/nfs/TemporalSummarization/ts15/results/updates_sampled.extended.tsv']

Number of streamids searching for: 53141


Previously processed 241 of 241 paths for topic 1



Previously processed 241 of 241 paths for topic 2



Previously processed 241 of 241 paths for topic 3



Previously processed 241 of 241 paths for topic 4


...

len matches: 0
saved proc_history
Processing file at: /nfs/trects-kba2014/45/2012-11-04-03.gz
len matches: 0
saved proc_history
Processing file at: /nfs/trects-kba2014/45/2012-11-01-12.gz
len matches: 11
File written to: /nfs/mine-trects-kba2014-filtered/45/2012-11-01-12.gz
saved proc_history
Processing file at: /nfs/trects-kba2014/45/2012-10-30-17.gz
len matches: 7
File written to: /nfs/mine-trects-kba2014-filtered/45/2012-10-30-17.gz
saved proc_history
Processing file at: /nfs/trects-kba2014/45/2012-10-29-16.gz
len matches: 41
File written to: /nfs/mine-trects-kba2014-filtered/45/2012-10-29-16.gz
saved proc_history
Processing file at: /nfs/trects-kba2014/45/2012-10-30-03.gz
len matches: 5
File written to: /nfs/mine-trects-kba2014-filtered/45/2012-10-30-03.gz
saved proc_history
Processing file at: /nfs/trects-kba2014/45/2012-11-01-21.gz
len matches: 9
File written to: /nfs/mine-trects-kba2014-filtered/45/2012-11-01-21.gz
saved proc_history
Processing file at: /nfs/trects-kba2014/45/20

len matches: 10
File written to: /nfs/mine-trects-kba2014-filtered/45/2012-10-30-06.gz
saved proc_history
Processing file at: /nfs/trects-kba2014/45/2012-11-02-23.gz
len matches: 2
File written to: /nfs/mine-trects-kba2014-filtered/45/2012-11-02-23.gz
saved proc_history
Processing file at: /nfs/trects-kba2014/45/2012-10-30-02.gz
len matches: 3
File written to: /nfs/mine-trects-kba2014-filtered/45/2012-10-30-02.gz
saved proc_history
Processing file at: /nfs/trects-kba2014/45/2012-11-03-23.gz
len matches: 1
File written to: /nfs/mine-trects-kba2014-filtered/45/2012-11-03-23.gz
saved proc_history
Processing file at: /nfs/trects-kba2014/45/2012-11-01-16.gz
len matches: 27
File written to: /nfs/mine-trects-kba2014-filtered/45/2012-11-01-16.gz
saved proc_history
Processing file at: /nfs/trects-kba2014/45/2012-11-06-15.gz
len matches: 0
saved proc_history
Processing file at: /nfs/trects-kba2014/45/2012-11-04-11.gz
len matches: 0
saved proc_history
Processing file at: /nfs/trects-kba2014/45/20

Processing file at: /nfs/trects-kba2014/46/2012-09-11-08.gz
len matches: 3
File written to: /nfs/mine-trects-kba2014-filtered/46/2012-09-11-08.gz
saved proc_history
Processing file at: /nfs/trects-kba2014/46/2012-09-11-06.gz
len matches: 1
File written to: /nfs/mine-trects-kba2014-filtered/46/2012-09-11-06.gz
saved proc_history
Processing file at: /nfs/trects-kba2014/46/2012-09-15-07.gz
len matches: 2
File written to: /nfs/mine-trects-kba2014-filtered/46/2012-09-15-07.gz
saved proc_history
Processing file at: /nfs/trects-kba2014/46/2012-09-15-05.gz
len matches: 14
File written to: /nfs/mine-trects-kba2014-filtered/46/2012-09-15-05.gz
saved proc_history
Processing file at: /nfs/trects-kba2014/46/2012-09-15-15.gz
len matches: 8
File written to: /nfs/mine-trects-kba2014-filtered/46/2012-09-15-15.gz
saved proc_history
Processing file at: /nfs/trects-kba2014/46/2012-09-12-10.gz
len matches: 9
File written to: /nfs/mine-trects-kba2014-filtered/46/2012-09-12-10.gz
saved proc_history
Processin

len matches: 11
File written to: /nfs/mine-trects-kba2014-filtered/46/2012-09-15-21.gz
saved proc_history
Processing file at: /nfs/trects-kba2014/46/2012-09-12-09.gz
len matches: 14
File written to: /nfs/mine-trects-kba2014-filtered/46/2012-09-12-09.gz
saved proc_history
Processing file at: /nfs/trects-kba2014/46/2012-09-14-04.gz
len matches: 3
File written to: /nfs/mine-trects-kba2014-filtered/46/2012-09-14-04.gz
saved proc_history
Processing file at: /nfs/trects-kba2014/46/2012-09-15-06.gz
len matches: 8
File written to: /nfs/mine-trects-kba2014-filtered/46/2012-09-15-06.gz
saved proc_history
Processing file at: /nfs/trects-kba2014/46/2012-09-14-11.gz
len matches: 23
File written to: /nfs/mine-trects-kba2014-filtered/46/2012-09-14-11.gz
saved proc_history
Processing file at: /nfs/trects-kba2014/46/2012-09-11-15.gz
len matches: 7
File written to: /nfs/mine-trects-kba2014-filtered/46/2012-09-11-15.gz
saved proc_history
Processing file at: /nfs/trects-kba2014/46/2012-09-13-16.gz
len mat

len matches: 8
File written to: /nfs/mine-trects-kba2014-filtered/46/2012-09-15-02.gz
saved proc_history
Processing file at: /nfs/trects-kba2014/46/2012-09-14-14.gz
len matches: 4
File written to: /nfs/mine-trects-kba2014-filtered/46/2012-09-14-14.gz
saved proc_history
Processing file at: /nfs/trects-kba2014/46/2012-09-15-09.gz
len matches: 12
File written to: /nfs/mine-trects-kba2014-filtered/46/2012-09-15-09.gz
saved proc_history
Processing file at: /nfs/trects-kba2014/46/2012-09-12-19.gz
len matches: 7
File written to: /nfs/mine-trects-kba2014-filtered/46/2012-09-12-19.gz
saved proc_history
Processing file at: /nfs/trects-kba2014/46/2012-09-15-12.gz
len matches: 2
File written to: /nfs/mine-trects-kba2014-filtered/46/2012-09-15-12.gz
saved proc_history
Processing file at: /nfs/trects-kba2014/46/2012-09-11-02.gz
len matches: 5
File written to: /nfs/mine-trects-kba2014-filtered/46/2012-09-11-02.gz
saved proc_history
Processing file at: /nfs/trects-kba2014/46/2012-09-12-08.gz
len match

In [129]:
# # test1_path = "/nfs/trects-kba2014-filtered-mine/1/2012-02-27-00.gz"
# # test2_path = "/nfs/trects-kba2014-filtered-mine/1/2012-02-26-15.gz"
# # test1_markup = open_markup_file(test1_path)
# # for doc in test1_markup.find_all("doc"):
# #     print(doc.find("zulu"))
# def get_stream_id(line, tag="streamid"):
#     start_tag = "<" + tag + ">"
#     end_tag = "</" + tag + ">"
#     no_tags = line.replace(start_tag,'').replace(end_tag,'')
#     return no_tags

# sidstr = "<streamid>1330304100-51d9dc9b383110fa1f802f80220ed4e1</streamid>"
# print(get_stream_id(sidstr))
# # print(sidstr.find("<streamid>"))
# # print(sidstr.replace('<streamid>','').replace('</streamid>',''))
# # # print(sidstr[10:])

</streamid>
1330304100-51d9dc9b383110fa1f802f80220ed4e1


## Control Generate and Load a Corpus

In [ ]:
class CorpusGenerator:
    def __init__(self, proj_dir, corpus_split_step=200):
        self.path_handler = FilePathHandler(proj_dir)
        # ["topics", "corpus", "nuggets", "embed_labels", "updates"]
        self.file_purposes = self.path_handler.file_purposes
        self.corpus_split_step = corpus_split_step
        self.topic_dfs = {}  # dict of topic dfs per corpus_name
        self.force_reload_options = ["topics", "corpus", "nuggets"]
        
    def generate(self, selection=None, corpus_names=None, new_corpuses=None, 
             force_reload=False, save=True, verbose=True):
        """
        Parameters:
            force_reload: force generate new dataframes if files already exist
                options: True, False or list of selection i.e. ["topics", "corpus", "nuggets"]
        """
        # interpret force_reload input
        if type(force_reload) is not list:
            if type(force_reload) is bool:
                if force_reload == True:
                    force_reload = self.force_reload_options
                else:
                    force_reload = []  # empty list, no chosen selection
        else:
            for select in force_reload:
                if select not in self.force_reload_options:
                    raise ValueError(str(select) + " is not a force_reload option")
        
        # add new corpuses to load
        if new_corpuses is not None:
            for new_corpus in new_corpuses:
                self.path_handler.add_corpus_source(new_corpus, overwrite=True)
        # get corpus paths to load from (if corpus_names is None loads all)
        self.corpus_sources = self.path_handler.get_corpus_sources(corpus_names=corpus_names)
        corpus_names = self.corpus_sources.keys()
        
        if selection is None:  # if none do all
            selection = self.file_purposes
        
        for corpus_name in corpus_names:
            print("corpus_name generate loop: " + str(corpus_name))
            if "topics" in selection:
                # create topic_df for corpus_name
                print("Generating topic_df")
                topic_reload = self.check_force_reload(corpus_name, "topics", force_reload, change_paths=True,
                                                      verbose=verbose)
                self.load_topic_df_control(corpus_name, save=save, force_reload=topic_reload, verbose=verbose,
                                          add_path=True)
                
            if "corpus" in selection:
                # create corpus df csvs
                print("Generating corpus_dfs")
                corp_reload = self.check_force_reload(corpus_name, "corpus", force_reload, change_paths=True,
                                                     verbose=verbose)
                self.corpus_splitter(corpus_name, force_reload=corp_reload, verbose=False)
                
            if "nuggets" in selection:
                print("Generating nugget files")
                nug_reload = self.check_force_reload(corpus_name, "nuggets", force_reload, change_paths=True,
                                                    verbose=verbose)
                self.nuggets_generator(corpus_name, force_reload=nug_reload, verbose=verbose)
        print("Finished generating files")
        
    def check_force_reload(self, corpus_name, select, force_reload, change_paths=True, inst_identifier=None,
                          verbose=True):
        if select in force_reload:
            if change_paths:
                self.change_force_reload_paths(corpus_name, select, inst_identifier=inst_identifier)
                if verbose:
                    print("Changed paths for " + str(select) + " in " + str(corpus_name))
            return True
        else:
            return False
        
    def change_force_reload_paths(self, corpus_name, select, inst_identifier=None):
        # change path df paths to not exists if force_reload
        targ = self.path_handler.path_df
        targ = targ[targ['corpus_name'] == corpus_name]
        targ = targ[targ['file_purpose'] == select]
        if inst_identifier is not None:
            targ = targ[targ['instance_identifier'] == inst_identifier]
        targ['exists'] = False  # set value for all items
        self.path_handler.save_path_df()
                    
            
    def nuggets_generator(self, corpus_name, force_reload=False, verbose=True):
        # if not exists load topics
        self.load_topic_df_control(corpus_name, force_reload=force_reload, verbose=verbose)
        
        # this is used for the filename of the resulting saved file
        base_identifier = "nuggets"
        
        topic_df = self.topic_dfs[corpus_name]
        topic_ids = list(topic_df['id'].unique())
        # create nuggets_df per topic
        for topic_id in tqdm_notebook(topic_ids):
            # get paths for corpus files for this topic_id (inelegantly)
            print("nuggets_generator topic_id: " + str(topic_id))
            corp_paths = self.path_handler.path_df
            corp_paths = corp_paths[corp_paths['corpus_name'] == corpus_name]
            corp_paths = corp_paths[corp_paths['file_purpose'] == 'corpus']
            inst_idents = corp_paths['instance_identifier'].unique()
            corp_paths = corp_paths[corp_paths['instance_identifier'] == str(topic_id)]  # paths for this topic
            corp_paths = corp_paths[corp_paths['exists'] == True]
            
            # check that all paths are loaded
            try:
                num_split = int(corp_paths.iloc[0]['num_splits'])
                if num_split < len(corp_paths):
                    raise IndexError()
            except IndexError:
                if verbose:
                    print(display(corp_paths))
                warnings.warn("Corpus files for topic " + str(topic_id) + " have not been fully loaded")
                continue
                
            corp_paths = list(corp_paths['path'])  # transform into actual list of paths
            
            # load corpus for a given topic
            corpus_df = load_corpus(corp_paths, save=False,
                                    force_reload=False, verbose=False, path_handler=self.path_handler)
            
            # nugget save destination
            save_path = self.path_handler.get_path(corpus_name, "nuggets", base_identifier, ".csv.gz",
                                        split_identifier=str(topic_id), num_splits=len(topic_ids), add_path=True)
            
            nuggets_tsv_path = self.corpus_sources[corpus_name]['nuggets_path']
            matches_tsv_path = self.corpus_sources[corpus_name]['matches_path']
            # generate nugget file
            load_nugget_df(save_path, corpus_df, topic_ids=[topic_id], matches_tsv_path=matches_tsv_path, 
                            nuggets_tsv_path=nuggets_tsv_path, save=True, force_reload=force_reload, 
                           verbose=verbose, path_handler=self.path_handler, spacy_if_not_found=True)
            
            
                    
    def corpus_splitter(self, corpus_name, force_reload=False, verbose=True):
        # split by topic and then every 200 html gz files, then parse together in loading
        # add check for what's been done already (i.e. check current topics, if all splits taken place)
        
        # if not exists load topics
        self.load_topic_df_control(corpus_name, force_reload=force_reload, verbose=verbose)
        topic_df = self.topic_dfs[corpus_name]
        
        corpus_dir = self.corpus_sources[corpus_name]["dir_path"]
        if verbose:
            print("corpus_dir:" + str(corpus_dir))
            
        for topic_id in topic_df['id'].unique():
            # confirm dir exists
            t_dir = corpus_dir + '/' + str(topic_id)
            if not file_exists(t_dir):
                warnings.warn("Corpus loading path at " + t_dir + " does not exist. Removing from topic_df")
                topic_df = self.remove_topic_and_save(corpus_name, topic_df, topic_id, verbose=verbose)
                continue
            
            # check if path exists
            p_df = self.path_handler.path_df
            p_df = p_df[p_df['corpus_name'] == corpus_name]
            p_df = p_df[p_df['file_purpose'] == "corpus"]
            t_df_paths = p_df[p_df['instance_identifier'] == str(topic_id)]
            t_df_paths = t_df_paths[t_df_paths['exists'] == True]  # only concerned with created files
            start_split = 0
            num_splits = 0
            if len(t_df_paths) == 0 or force_reload:  # not yet processed
                start_split = 0
            else:
                # check if all splits been processed
                num_splits = list(t_df_paths['num_splits'])[0]  # ensure same num_splits is inputted into path_df
                if len(t_df_paths) < num_splits:  # not counting from zero
                    break
                # get start point if partway through
                start_split = max(list(map(int, list(t_df_paths['split_identifier']))))
            
            num_files = len(search_dir(t_dir))
            if len(num_files) == 0:
                warnings.warn("No files found in directory " + str(t_dir) + ". Removing " + str(topic_id)
                             + " from topic_df")
                topic_df = self.remove_topic_and_save(corpus_name, topic_df, topic_id, verbose=verbose)
            
            # create split indexes to feed to load_corpus
            splits = [start_split]
            add = splits[-1] + self.corpus_split_step
            while add < num_files:
                splits.append(int(add))
                add = splits[-1] + self.corpus_split_step
            
            if start_split == 0:  
                num_splits = int(len(splits))  # for inputting into path_df
            
            if verbose:
                print("creating corpus df for topic " + str(topic_id) + " starting at file no. " 
                      + str(split_start_doc) + " of " + str(num_files) + " splitting every " 
                      + str(split_every) + " files")
            # create corpus_df files
            for split_num in splits:
                # get save path
                save_path = self.path_handler.get_path(corpus_name, "corpus", str(topic_id), ".csv.gz",
                                        split_identifier=str(split_num), num_splits=num_splits, add_path=True)
                
                load_corpus(save_path, corpus_dir=corpus_dir, topic_ids=[topic_id], 
                            split_every=self.corpus_split_step, split_start_doc=split_num, 
                            save=True, force_reload=force_reload, 
                            verbose=verbose, path_handler=self.path_handler)
    
    def remove_topic_and_save(self, corpus_name, topic_df, topic_id, verbose=True):
        # remove topic_id from topic_df and save
        topic_df = topic_df[topic_df['id'] != topic_id]
        path = self.path_handler.path_df
        path = path[(path['corpus_name'] == corpus_name) & (path['file_purpose'] == "topics")]['path']
        path = list(path)[0]
        topic_df.to_csv(path, compression='gzip')
        if verbose:
            print(str(topic_id) + " remoed from topic_df and saved to " + str(path))
        return topic_df
                
    def load_topic_df_control(self, corpus_name, save=True, force_reload=False, verbose=True, add_path=False):
        if self.topic_dfs is None:
            self.topic_dfs = {}
        if corpus_name not in self.topic_dfs:
            self.topic_dfs[corpus_name] = self.load_topic_df(corpus_name, save=save, force_reload=force_reload,
                                                             verbose=verbose, add_path=add_path)
                

    def load_topic_df(self, corpus_name, save=True, force_reload=False, verbose=True, add_path=False):
        load_path = self.corpus_sources[corpus_name]["topics_path"]
        save_path = self.path_handler.get_path(corpus_name, "topics", "topics_df", ".csv.gz", add_path=add_path)
        
        topic_df = load_topics(save_path, load_path=load_path, save=save, force_reload=force_reload, 
                               verbose=verbose, path_handler=self.path_handler)
        return topic_df

In [329]:
proj_dir = '/nfs/proj-repo/AAARG-dissertation'
orig_tr14_filtered_dict = { "corpus_name":"original-trects-kba2014-filtered",
                        "dir_path":"/nfs/original-trects-kba2014-filtered", 
                      "topics_path":"/nfs/original-trects-kba2014-filtered/test-topics.xml", 
                      "nuggets_path":"/nfs/TemporalSummarization/ts13/results/nuggets.tsv",
                        "matches_path":"/nfs/TemporalSummarization/ts13/results/matches.tsv"}

mine_tr14_filtered_dict = {"corpus_name":"mine-trects-kba2014-filtered",
                          "dir_path":"/nfs/mine-trects-kba2014-filtered",
                          "topics_path":["/nfs/TemporalSummarization/ts13/test-topics.xml",
                                        "/nfs/TemporalSummarization/ts14/trec2014-ts-topics-test.xml",
                                        "/nfs/TemporalSummarization/ts15/trec2015-ts-topics-test.xml"],
                          "nuggets_path":["/nfs/TemporalSummarization/ts13/results/nuggets.tsv",
                                         "/nfs/TemporalSummarization/ts14/results/nuggets.tsv",
                                         "/nfs/TemporalSummarization/ts15/results/nuggets.tsv"],
                          "matches_path":["/nfs/TemporalSummarization/ts13/results/matches.tsv",
                                         "/nfs/TemporalSummarization/ts14/results/matches.tsv",
                                         "/nfs/TemporalSummarization/ts15/results/matches.tsv"]}

# tr14_filtered_mine = {"corpus_name":"tr14_filtered_mine",
#                      "dir_path":"/nfs/trects-kba2014-filtered-mine",
#                      "topics_path":}

# nlp = spacy.load("en_core_web_sm")
corp_gen = CorpusGenerator(proj_dir)

force_reload = ["nuggets"]
# force_reload = False
selection = ["nuggets"]
corpus_names = ["mine-trects-kba2014-filtered"]

corp_gen.generate(new_corpuses=[orig_tr14_filtered_dict, mine_tr14_filtered_dict], corpus_names=corpus_names, 
                  force_reload=force_reload, verbose=True, selection=selection)
p_han = FilePathHandler(proj_dir)
print(display(p_han.path_df[0:10]))

corpus_name generate loop: mine-trects-kba2014-filtered
Generating nugget files
Changed paths for nuggets in mine-trects-kba2014-filtered
Loading topics
0 duplicate documents removed from topics df
df loaded
saved at: /nfs/proj-repo/AAARG-dissertation/dataset/mine-trects-kba2014-filtered/topics/topics_df.csv.gz


,id,title,description,start,end,query,type
0,1,2012 Buenos Aires Rail Disaster,http://en.wikipedia.org/wiki/2012_Buenos_Aires...,1329910380,1330774380,buenos aires train crash,accident
1,2,2012 Pakistan garment factory fires,http://en.wikipedia.org/wiki/2012_Pakistan_gar...,1347368400,1348232400,pakistan factory fire,accident
2,3,2012 Aurora shooting,http://en.wikipedia.org/wiki/2012_Aurora_shooting,1342766280,1343630280,colorado shooting,shooting
3,4,Wisconsin Sikh temple shooting,http://en.wikipedia.org/wiki/Wisconsin_Sikh_te...,1344180300,1345044300,sikh temple shooting,shooting


None


nuggets_generator topic_id: 1
Loading nugget_df


  0%|          | 0/1172 [00:00<?, ?it/s]

...

Nugget entries were generated for 304 nuggets. There were 2 found in matches.tsv but not in corpus
There were 5 nugget_ids found in matches.tsv but not in nuggets.tsv
5 out of 304 streamids had out of bounds sent_ids
nugget_df entries: 304
df loaded
saved at: /nfs/proj-repo/AAARG-dissertation/dataset/mine-trects-kba2014-filtered/nuggets/nuggets_46.csv.gz


,topic_id,match_start,match_end,importance,nugget_len,nugget_text,sent_in_text,sent_id,docid,streamid,epoch
0,46.0,24.0,69.0,2.0,100.0,The number of people taking part in the demons...,1. Barcelona braces for million-strong march f...,234.0,529c12d25c4bb1b9bf66f0e2e8f0ca26,1347352633-529c12d25c4bb1b9bf66f0e2e8f0ca26,1.347353e+09
1,46.0,0.0,66.0,2.0,100.0,The number of people taking part in the demons...,Barcelona braces for million-strong march for ...,814.0,a69d3d2d776ded792f5f6985acaf398b,1347369485-a69d3d2d776ded792f5f6985acaf398b,1.347369e+09
2,46.0,1.0,204.0,1.0,92.0,The financial crisis has strained the already ...,The lingering effects of the global financial ...,32.0,be52a6b7690622e7ffb5fc82928ae889,1347372452-be52a6b7690622e7ffb5fc82928ae889,1.347372e+09
3,46.0,378.0,506.0,3.0,105.0,The 2010 demonstration supporting Catalan in...,"and now more than ever , '' Mr Mas , of the na...",42.0,c3ec41c3aec064d4214a63fd57ab3e27,1347380912-c3ec41c3aec064d4214a63fd57ab3e27,1.347381e+09


None

Finished generating files


,corpus_name,file_purpose,split_identifier,num_splits,instance_identifier,file_type,path,exists
0,original-trects-kba2014-filtered,topics,NaN,1,topics_df,.csv.gz,/nfs/proj-repo/AAARG-dissertation/dataset/orig...,True
1,original-trects-kba2014-filtered,corpus,0.0,2,1,.csv.gz,/nfs/proj-repo/AAARG-dissertation/dataset/orig...,True
2,original-trects-kba2014-filtered,corpus,200.0,2,1,.csv.gz,/nfs/proj-repo/AAARG-dissertation/dataset/orig...,True
3,original-trects-kba2014-filtered,corpus,0.0,2,2,.csv.gz,/nfs/proj-repo/AAARG-dissertation/dataset/orig...,True
4,original-trects-kba2014-filtered,corpus,200.0,2,2,.csv.gz,/nfs/proj-repo/AAARG-dissertation/dataset/orig...,True
5,original-trects-kba2014-filtered,corpus,0.0,2,3,.csv.gz,/nfs/proj-repo/AAARG-dissertation/dataset/orig...,True
6,original-trects-kba2014-filtered,corpus,200.0,2,3,.csv.gz,/nfs/proj-repo/AAARG-dissertation/dataset/orig...,True
7,original-trects-kba2014-filtered,corpus,0.0,2,4,.csv.gz,/nfs/proj-repo/AAARG-dissertation/dataset/orig...,True
8,original-trects-kba2014-filtered,corpus,200.0,2,4,.csv.gz,/nfs/proj-repo/AAARG-dissertation/dataset/orig...,True
9,original-trects-kba2014-filtered,corpus,0.0,2,5,.csv.gz,/nfs/proj-repo/AAARG-dissertation/dataset/orig...,True


None


In [ ]:
# class CorpusLoader:
#     def __init__(self):
#         self.path_handler = FilePathHandler()
#         self.file_purposes = self.path_handler.file_purposes
        
        
#     def load(self, selection=None, corpus_names=None, new_corpuses=None, force_reload=False, save=True, 
#              create_only=False, verbose=True):
#         """
#         Parameters:
#             selection: the data to load (e.g. corpus/nuggets), if None then load options
#             corpus_names: list of corpus names to load
#             new_corpuses: list of dicts of paths with keys {"dir_path", "topics_path", "nuggets_path"}
#             force_reload: force rebuild corpus files from original files
#             save: save built corpus files
#             create_only: only generate missing corpus files, do not load into ram
#         """
#         """Put flags in here to control process too
#         Steps:
#         1. Load corpus from gz html files
#         2. Load topics from topics file
#         3. Create nuggets_df from nuggets file
#         4. Create embeddings from nuggets and corpus
#         """
#         # add new corpuses to load
#         if new_corpuses is not None:
#             for new_corpus in new_corpuses:
#                 self.path_handler.add_corpus(new_corpus, overwrite=False)
#         # get corpus paths to load from (if corpus_names is None loads all)
#         corpus_sources = self.path_handler.get_corpus_sources(corpus_names=corpus_names)
#         corpus_names = corpus_sources.keys()
        
#         # get paths for generated files in corpus
#         if selection is None:  # if None selection get all
#             selection = self.file_purposes
#         corpus_names_paths = {}
#         for corpus_name in corpus_names:
#             name_paths = self.path_handler.paths_in_corpus_name(corpus_name, selection=selection)
#             corpus_names_paths[corpus_name] = name_paths
        
#         # go through each selected corpus_name
#         for corpus_name, corpus_paths in corpus_names_paths.items():
#             # go through selected tasks
#             for select in selection:  # maybe add tqdm here?
#                 # for each identifier
#                 # may behave differently when no entries in paths_df
#                 for identifier, ident_paths in corpus_paths[select]:
#                     exists = ident_paths['exists']
#                     not_exists = ident_paths['not_exists']
#                     if exists is None or force_reload:  # no paths loaded
#                         # create all appropriate files
# #                         not_exists = self.path_handler.convert_relative_path(not_exists)
#                         self.select_load_func(select, paths=not_exists, save=save, 
#                                     force_reload=force_reload, verbose=verbose, create_only=create_only)
#                     else:
#                         if not_exists is None:  # only exists has paths
#                             if create_only:
#                                 # change this to something better
#                                 warnings.warn("There are no new files to create")
#                             else:
# #                                 exists = self.path_handler.convert_relative_path(exists)
#                                 self.select_load_func(select, paths=exists, save=save, 
#                                     force_reload=force_reload, verbose=verbose, create_only=create_only)
#                         else:  # both have paths
#                             # need method to load/generate incomplete missing parts
#                             # self.path_handler.convert_relative_paths(...) dont forget
    
#     def fix_partially_missing_paths(self, selection):
#         """Function to organise missing paths from partially-saved/generated dataset"""
#         if selection == "topics":
#             raise ValueError("There can only be one topics file for a corpus")
    
#     def select_load_func(self, selection, paths=None, save=True, force_reload=False, 
#                              verbose=True, create_only=False, **identifiers):
#         # ["topics", "corpus", "nuggets", "embed_labels", "updates"]
#         if paths is None:
#             # create paths appropriately, add to paths_df
#             # also check paths/behaviour is correct in each load_func/after
#             # also need to account for other identifiers
#         if selection == "topics":
#             # should only need to be one meta topics_df
#             topic_dfs = []
#             # need to add save paths into paths_df
#             topic_df = load_topics(paths[0], save=save, force_reload=force_reload, verbose=verbose)
                
#         elif selection == "corpus":
            
#         elif selection == "nuggets":
            
#         elif selection == "embed_labels":
            
#         elif selection == "updates":
        
        
        

In [123]:
# def test_updates_tsv():
#     def parse_update_id(update_id):
#         streamid = "-".join(update_id.split("-")[:-1])
#         return streamid
        
#     updates_path = "/nfs/TemporalSummarization/ts14/results/updates_sampled.tsv"
#     updates_df = pd.read_csv(updates_path, "\t")
#     test_corpus_path = "/nfs/proj-repo/AAARG-dissertation/l_d/corpus_loaded.csv.gz"
#     test_cdf = load_corpus(test_corpus_path, verbose=False)
#     test_cdf = test_cdf.drop_duplicates(subset=['streamid'])
    
#     print(display(updates_df[updates_df['update_id'].str.contains("1329908400-f65b681fc854df85fd15e53cbb988df8")]))
    
    
#     for index, row in tqdm_notebook(updates_df.iterrows()):
#         # find where in corpus streamid is
#         up_streamid = parse_update_id(row['update_id'])
# #         print(up_streamid)
#         up_sentid = row['sentence_id']
#         nug = test_cdf[test_cdf["streamid"] == up_streamid]
#         if len(nug) == 0:
#             # see if docid match
#             up_docid = row['doc_id']
#             nug = test_cdf[test_cdf['docid'] == up_docid]
# #             if len(nug) == 0:
# #                 print("no docid match")
# #             print("no streamid match")
#             continue
#         elif len(nug) > 1:
#             print("multiple streamid matches??")
#             print(display(nug))
# #             outstr = []
# #             for sid in nug['streamid']:
# #                 outstr.append(sid)
# #             print("streamids: " + ",".join(outstr))
#             continue
#         nug_text = str(nug['text'].iloc[0])
#         nug_text = nug_text.splitlines()
#         up_text = row['update_text']
# #         print("nug_text len: " + str(len(nug_text)))
#         for sent in nug_text:
#             if sent == up_text:
#                 print("streamid:" + up_streamid)
#                 print("Match:\n" + "sent: " + sent + "\n" + "up_text: " + up_text + "\n")
# #         nug_text = nug_text[up_sentid + 1]
# #         up_text = row['update_text']
# #         if up_text == nug_text:
# #             print("It's the same")
#     print("end function")
    
# test_updates_tsv()

In [170]:
# !python -m spacy download en_core_web_lg
# import en_core_web_lg

In [186]:
def check_matches_nuggets():
    def parse_update_id(update_id):
        update_id = update_id.split("-")
        sent_id = int(update_id[-1])
        streamid = "-".join(update_id[:-1])
        return streamid, sent_id
    matches_path = "/nfs/TemporalSummarization/ts13/results/matches.tsv"
    nuggets_path = "/nfs/TemporalSummarization/ts13/results/nuggets.tsv"
    test_corpus_path = "/nfs/proj-repo/AAARG-dissertation/l_d/corpus_loaded.csv.gz"
    matches_df = pd.read_csv(matches_path, "\t")
    nugget_df = pd.read_csv(nuggets_path, "\t")
    corpus_df = load_corpus(test_corpus_path, verbose=False)
    corpus_df = corpus_df.drop_duplicates(subset=['streamid'])
    
#     nlp = spacy.load("en_core_web_lg")  # spacy char sent matching
    nlp = en_core_web_lg.load()
    print("loaded spacy model")
    
    for index, row in matches_df.iterrows():
        streamid, sent_id = parse_update_id(row['update_id'])
#         sent_id += 1
        
        c_sid = corpus_df[corpus_df['streamid'] == streamid]
        
        if len(c_sid) == 0:
            continue
        
        c_sid_str = str(c_sid['text'].iloc[0])
        nug_text = str(nugget_df[nugget_df['nugget_id'] == row['nugget_id']].iloc[0]['nugget_text'])
        
        c_sid_text = c_sid_str.splitlines()
        if c_sid_text[0] == "":
            sent_id += 1
        match_start = row['match_start']
        match_end = row['match_end']
        bound_sent = c_sid_str[match_start:match_end + 1]  # char indexing, start:end
#         print("sent_id: " + str(sent_id))  # sent id from update_id
        targ_sent = None
        try:
            targ_sent = c_sid_text[sent_id]  # sent_id from update_id sentence
        except IndexError as e:
            print("index error, sent_id/len: " + str(sent_id) + " / " + str(len(c_sid_text)))
        contains = False
        if c_sid_str.find(nug_text) != -1:
            contains = True
        char_count = 0
        bound_target_sent = None  # char indexing, using only match_start and matching sentence lengths
        for senten in c_sid_text:
            s_len = len(senten)
            if char_count + s_len > match_start:
                bound_target_sent = senten
                break
            char_count += s_len
            
        spacy_text = nlp(c_sid_str)
        spacy_sents = list(spacy_text.sents)
        bound_targ_sent_spacy = None
        char_count = 0
        for s in spacy_sents:
            s = str(s)
            s_len = len(s)
            if char_count + s_len > match_start:
                bound_targ_sent_spacy = s
                break
            char_count += s_len
            
        print("nug_text: " + str(nug_text) + "\n")
        print("sent_id (" + str(sent_id) + "): " + str(targ_sent) + "\n")
        print("bound_sent: " + str(bound_sent) + "\n")
        print("bound_targ_sent: " + str(bound_target_sent) + "\n")
        print("bound_targ_sent_spacy: " + str(bound_targ_sent_spacy) + "\n")
        print("contains: " + str(contains) + "\n")
        print("---------------------------------------------------------------")

check_matches_nuggets()    

loaded spacy model
nug_text: 676+ injuries

sent_id (1): Buenos Aires Train Crash Kills 49, Injuries over 600.

bound_sent:  Injuries over 600.

bound_targ_sent: Buenos Aires Train Crash Kills 49, Injuries over 600.

bound_targ_sent_spacy: 
Buenos Aires Train Crash Kills 49, Injuries over 600.


contains: False

---------------------------------------------------------------
nug_text: train accident in Buenos Aires, Argentina.

sent_id (3): "The train was full and the impact was 49 dead in Buenos Aires train crash: police.

bound_sent: is is a Smart Blog and Wr

bound_targ_sent: Argentina train crash in Buenos Aires kills 49 This is a Smart Blog and Write All About Home Improvement & Health Care By Michael Warren | AP BUENOS AIRES , Argentina ( AP) — A packed train slammed into the end of the line in Buenos Aires ' busy Once station today, killing 49 people and injuring hundreds of morning commuters as passenger cars crumpled behind the engine.

bound_targ_sent_spacy: This is a Smart B

nug_text: 49 confirmed deaths

sent_id (3): "The train was full and the impact was 49 dead in Buenos Aires train crash: police.

bound_sent: kills 49

bound_targ_sent: Argentina train crash in Buenos Aires kills 49 This is a Smart Blog and Write All About Home Improvement & Health Care By Michael Warren | AP BUENOS AIRES , Argentina ( AP) — A packed train slammed into the end of the line in Buenos Aires ' busy Once station today, killing 49 people and injuring hundreds of morning commuters as passenger cars crumpled behind the engine.

bound_targ_sent_spacy: 
Argentina train crash in Buenos Aires kills 49

contains: False

---------------------------------------------------------------
nug_text: train accident in Buenos Aires, Argentina.

sent_id (3): Deborah Lutterbeck AP BUENOS AIRES — A train packed with morning commuters slammed into a barrier at the end of a line Wednesday, killing 49 people and injuring hundreds as passenger cars crumpled and windows exploded.

bound_sent:  in Bu

nug_text: the train crashed at the buffer stop

sent_id (3): Deborah Lutterbeck AP BUENOS AIRES — A train packed with morning commuters slammed into a barrier at the end of a line Wednesday, killing 49 people and injuring hundreds as passenger cars crumpled and windows exploded.

bound_sent: kills 49 The Daily News - Reading Everything Under The World Buenos Aires train c

bound_targ_sent: Argentina train crash in Buenos Aires kills 49 The Daily News - Reading Everything Under The World Buenos Aires train crash leaves at least 40 dead and over 500 more injured.

bound_targ_sent_spacy: 
Argentina train crash in Buenos Aires kills 49

contains: False

---------------------------------------------------------------
nug_text: Hundreds injured

sent_id (3): Deborah Lutterbeck AP BUENOS AIRES — A train packed with morning commuters slammed into a barrier at the end of a line Wednesday, killing 49 people and injuring hundreds as passenger cars crumpled and windows exploded.

bound_sent:  over

nug_text: train accident in Buenos Aires, Argentina.

sent_id (3): ( Xinhua/Telenoticiosa Americana ) At least one Chinese national was injured in a rush hour train crash that had killed 49 people and injured more than 600 in Argentina 's capital Buenos Aires on Wednesday, according to the Chinese Embassy .

bound_sent: ch Home China World Business Life Entertainment Sports Insight Photo Gallery Romney wins GOP caucus i

bound_targ_sent: At least 1 Chinese injured in Argentina train crash_ Top Stories -- China Economic Net Search Home China World Business Life Entertainment Sports Insight Photo Gallery Romney wins GOP caucus in Guam US , ROK , Japan to meet over talks with Pyongyang IMF chief to propose 28-bln - euro loan for Greece Greek debt swap plan participation rate up to 95.7 pct Greece gets 85.8% takeup in swap CDB extends loans to vegetable transportation China posts Feb. trade deficit as imports jump February auto sales rise 24.5% EU to keep carbon tax Lagarde to visit China 

nug_text: the train crashed at the buffer stop

sent_id (6): Transportation Secretary Juan Pablo Schiavi said the train entered the station at a speed of 20km/h ) and failed to stop, crashing into a retaining wall at the end of the track.

bound_sent: ry 23, 2012 | Filed underWorld | Posted by Video bueno

bound_targ_sent: Buenos Aires Train Crash, 49 killed and 600 injured Daily News Buenos Aires Train Crash , 49 killed and 600 injured February 23, 2012 | Filed underWorld | Posted by Video buenos aires train crash The first 2 cars were packed as usual for the morning rush, so tightly that people stood pressed flesh to flesh, sandwiched between bicycles and the few seats, many without so much as a strap to hold onto.

bound_targ_sent_spacy: 
Buenos Aires Train Crash, 49 killed and 600 injured Daily News Buenos Aires Train Crash , 49 killed and 600 injured February 23, 2012

contains: False

---------------------------------------------------------------
nug_text: 676+ injuries

sent_id

...

nug_text: Tel Aviv bus bombing

sent_id (1): Israel arrests suspects in Tel Aviv bus bombing | UnFox News Login | Create a free account Home World US Science Technology Green Politics Atheism Religion News Search Israel arrests suspects in Tel Aviv bus bombing JERUSALEM ( Reuters ) - Israeli authorities arrested an Israeli Arab on suspicion of planting a bomb in a Tel Aviv bus that wounded 15 people hours before Israel agreed a ceasefire with Hamas in Gaza , police and security officials said on Thursday.... read more... Share | Published By: Chicago Tribune - Today Older News Tel Aviv bus bombing shatters any illusions of safety LA Times World (Yesterday) - Attack leaves 21 injured in the Israeli city, which has enjoyed a relative sense of security in recent years. TEL AVIV — A bus bombing in downtown Tel Aviv , the first... Israel arrests suspects in Tel Aviv bus bombing Black Elk Order To Improve Safety After Rig Explosion Regulator warns Black Elk Energy on safety after rig fire Bo

nug_text: Terror suspects arrested

sent_id (1): Israel arrests suspects in Tel Aviv bus bombing | UnFox News Login | Create a free account Home World US Science Technology Green Politics Atheism Religion News Search Israel arrests suspects in Tel Aviv bus bombing JERUSALEM ( Reuters ) - Israeli authorities arrested an Israeli Arab on suspicion of planting a bomb in a Tel Aviv bus that wounded 15 people hours before Israel agreed a ceasefire with Hamas in Gaza , police and security officials said on Thursday.... read more... Share | Published By: Chicago Tribune - Today Older News Tel Aviv bus bombing shatters any illusions of safety LA Times World (Yesterday) - Attack leaves 21 injured in the Israeli city, which has enjoyed a relative sense of security in recent years. TEL AVIV — A bus bombing in downtown Tel Aviv , the first... Israel arrests suspects in Tel Aviv bus bombing Black Elk Order To Improve Safety After Rig Explosion Regulator warns Black Elk Energy on safety after rig fir

nug_text: Terror suspects arrested

sent_id (2): Secretary of State Hillary Clinton to pursue an elusive truce between... Israel - Hamas talks leave future of Gaza blockade cloudy Israel arrests suspects in Tel Aviv bus bombing Gaza ceasefire holds but mistrust runs deep Tel Aviv bus hit by bomb, at least 10 wounded Chicago Tribune (Yesterday) - TEL AVIV ( Reuters ) - A bomb exploded on a bus in central Tel Aviv on Wednesday, wounding at least 10 people in what Israeli officials said was a terrorist attack... Israel arrests suspects in Tel Aviv bus bombing Tel Aviv bus bombing shatters any illusions of safety Bomb blast hits bus in Tel Aviv Oil up to above $87 after Tel Aviv bus explosion Seattle times Tech News (Yesterday) - The price of oil rebounded slightly to above $87 a barrel on Wednesday after an explosion injured 10 people on a bus in Tel Aviv , a development that has the... Israel arrests suspects in Tel Aviv bus bombing Tel Aviv bus bombing shatters any illusions of safety B

nug_text: occured in the heart of Tel Aviv near military hdqtrs

sent_id (115): The bus was reportedly passing the military headquarters in the city at the time of the blast.

bound_sent: 
Israel says it arrests Tel Aviv bus bomber - Worldnews.com Explore WN Photos Travel Movies Phot

bound_targ_sent: Israel says it arrests Tel Aviv bus bomber - Worldnews.com Explore WN Photos Travel Movies Photographers Health Science Technology Cities Live TV World News Login Edit Israel says it arrests Tel Aviv bus bomber Tweet sms this page email this page News Videos Video Details Location Images Related Links Twitter BBC Results Shopping Travel Booking The News &amp ; Observer 2012-11-22 : Dan Balilty - AP Photo Israeli police and security personnel stand next to a destroyed bus at the site of a bombing in Tel Aviv , Israel , Wednesday, Nov. 21, 2012 .

bound_targ_sent_spacy: 
Israel says it arrests Tel Aviv bus bomber - Worldnews.com

contains: False

--------------------------------------------

IndexError: single positional indexer is out-of-bounds

In [99]:
def test_non_standard_topic_id():
    def generate_update_paths():
        up_dir = "/nfs/TemporalSummarization"
        ts_dirs = ["ts13", "ts14", "ts15"]
        target_files = ['updates_sampled.extended.tsv', 'updates_sampled.tsv',
                                 'matches.tsv', 'nuggets.tsv']
        
        up_paths = []
        for ts_dir in ts_dirs:
            for target_file in target_files:
                full_path = up_dir + '/' + ts_dir + '/results/' + target_file
                if os.path.exists(full_path):
                    up_paths.append(full_path)
                else:
                    print("doesn't exist: " + str(full_path))
        return up_paths
    
    def get_num_topic_id(topic_id):
        split = topic_id.split(".", 2)  # e.g. 'TS14.18'
        try:
            if split[0].upper() == "TS14":
                topic_id = int(split[1])  # extract int '18'
                return topic_id
            else:
                raise ValueError()
        except ValueError:
            print("can't convert " + str(topic_id))
            return None  # no discernable topic_id
    
    tsv_paths = generate_update_paths()
    topic_ids = set()
    
    
    
    for tsv_path in tsv_paths:
        try:
            tsv_df = pd.read_csv(tsv_path, "\t")
#             print(str(tsv_path) + "\ntopic_ids: " + str(tsv_df['query_id'].unique()))
            if 'query_id' in tsv_df.columns:
                query_ids = set(list(tsv_df['query_id']))
                topic_ids.update(query_ids)
        except FileNotFoundError:
            print("not found tsv_path: " + str(tsv_path))
            
    non_standard = []
    for topic_id in topic_ids:
        try:
            conv = int(topic_id)
        except ValueError:
            non_standard.append(topic_id)
    print("non_standard")
    print(non_standard)
    
    converted = []
    for non in non_standard:
        tid = get_num_topic_id(non)
        if tid is not None:
            converted.append(tid)
    print("converted")
    print(converted)
    
    print("finished")
    
#     print(topic_ids)
    
test_non_standard_topic_id()

doesn't exist: /nfs/TemporalSummarization/ts13/results/updates_sampled.extended.tsv
doesn't exist: /nfs/TemporalSummarization/ts15/results/updates_sampled.extended.tsv
non_standard
['TS14.14', 'TS14.25', 'TS14.20', 'TS14.22', 'TS14.13', 'TS14.15', 'TS14.24', 'TS14.21', 'TS14.16', 'TS13.07', 'TS14.19', 'TS13.TEST01', 'TS14.11', 'TS14.12', 'TS14.18', 'TS14.23', 'TS14.17', 'TS13.18', 'TS13.13']
can't convert TS13.07
can't convert TS13.TEST01
can't convert TS13.18
can't convert TS13.13
converted
[14, 25, 20, 22, 13, 15, 24, 21, 16, 19, 11, 12, 18, 23, 17]
finished
